In [52]:
! pip install requests
! pip install pillow
import requests
import os, sys
sys.path.insert(0,"..")
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import argparse
import skimage, skimage.io
import pprint

import warnings
from numpy import ndarray
from torchxrayvision.utils import normalize

import torch
import torch.nn.functional as F
import torchvision, torchvision.transforms

import torchxrayvision as xrv

from PIL import Image

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [86]:
def read_xray_dcm(path: os.PathLike) -> ndarray:
    """read a dicom-like file and convert to numpy array 

    Args:
        path (PathLike): path to the dicom file

    Returns:
        ndarray: 2D single array image for a dicom image scaled between -1024, 1024
    """
    try:
        import pydicom
    except ImportError:
        raise Exception("Missing Package Pydicom. Try installing it by running `pip install pydicom`.")

    # get the pixel array
    ds = pydicom.dcmread(path, force=True)

    # we have not tested RGB, YBR_FULL, or YBR_FULL_422 yet.
    if ds.PhotometricInterpretation not in ['MONOCHROME1', 'MONOCHROME2']:
        raise NotImplementedError(f'PhotometricInterpretation `{ds.PhotometricInterpretation}` is not yet supported.')

    data = ds.pixel_array
    
    # LUT for human friendly view
    data = pydicom.pixel_data_handlers.util.apply_voi_lut(data, ds, index=0)

    # `MONOCHROME1` have an inverted view; Bones are black; background is white
    # https://web.archive.org/web/20150920230923/http://www.mccauslandcenter.sc.edu/mricro/dicom/index.html
    if ds.PhotometricInterpretation == "MONOCHROME1":
        warnings.warn(f"Coverting MONOCHROME1 to MONOCHROME2 interpretation for file: {path}. Can be avoided by setting `fix_monochrome=False`")
        data = data.max() - data

    # normalize data to [-1024, 1024]
    data = normalize(data, data.max())
    return data


In [90]:
folder_path = '/Users/thiagohata/Documents/dicom_samples'

#def walk_in_folder(folder_path):
#    count = 0
#    for root, dirs, files in os.walk(folder_path):
#        for file_name in files:
#            if file_name.endswith(".dcm"):  # You can adjust this to detect DICOM files without extensions
#                file_path = os.path.join(root, file_name)
#                count+=1
#                #print(count)
#                #upload_dicom_file(file_path, orthanc_url)
#                return(file_path)

def walk_in_folder(folder_path):
    count = 0
    dicom_arrays = []  # List to store processed DICOM arrays

    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith(".dcm"):  # Detecting DICOM files by extension
                file_path = os.path.join(root, file_name)
                count += 1
                #print(f"Processing file {count}: {file_path}")
                
                # Call the read_xray_dcm function to process the DICOM file
                dicom_array = read_xray_dcm(file_path)
                
                # Add the processed array to the list
                dicom_arrays.append(dicom_array)
    
    # Return the list of all processed DICOM arrays
    return dicom_arrays

# images_list as a numpy_array
dicom_images = walk_in_folder(folder_path)

In [95]:

#pil_img_list = []
## Convert list of NumPy array to a Pillow Image
#def convert_array(dicom_images):
#    for i in range(len(dicom_images)):
#        image = Image.fromarray(dicom_images[i])
#
#        img = skimage.io.imread(image)
#        img = xrv.datasets.normalize(img, 255) 
#        # Check that images are 2D arrays
#        if len(img.shape) > 2:
#            img = img[:, :, 0]
#        if len(img.shape) < 2:
#            print("error, dimension lower than 2 for image")
#        
#        # Add color channel
#        img = img[None, :, :]
#        pil_img_list.append(img)
#    return(pil_img_list)
#convert_array()

In [101]:
#!/usr/bin/env python
# coding: utf-8
pil_img_list = []
def convert_array(dicom_images):
    for i in range(len(dicom_images)):
        image = Image.fromarray(dicom_images[i])
        pil_img_list.append(image)
    return(pil_img_list)

convert_array(dicom_images)

def process_image(pil_image):
    parser = argparse.ArgumentParser()
    parser.add_argument('-f', type=str, default="", help='')
    #parser.add_argument('img_path', type=str)
    parser.add_argument('-weights', type=str,default="densenet121-res224-all")
    parser.add_argument('-feats', default=False, help='', action='store_true')
    parser.add_argument('-cuda', default=False, help='', action='store_true')
    parser.add_argument('-resize', default=False, help='', action='store_true')

    cfg = parser.parse_args()

    #img = skimage.io.imread("/Users/thiagohata/Downloads/dicom_samples/id_0a0c2c8f-a36a1e82-a4857225-5a2af2a6-c7be16c1/Study_12840378.32185825.64169999.71049659.46899097/Series_60731327.33236805.18319358.84233616.48423037")

    img = skimage.io.imread(pil_image)
    img = xrv.datasets.normalize(img, 255)  

    # Check that images are 2D arrays
    if len(img.shape) > 2:
        img = img[:, :, 0]
    if len(img.shape) < 2:
        print("error, dimension lower than 2 for image")

    # Add color channel
    img = img[None, :, :]

    # the models will resize the input to the correct size so this is optional.
    if cfg.resize:
        transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),
                                                    xrv.datasets.XRayResizer(224)])
    else:
        transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop()])

    img = transform(img)


    model = xrv.models.get_model(cfg.weights)

    output = {}
    with torch.no_grad():
        img = torch.from_numpy(img).unsqueeze(0)
        if cfg.cuda:
            img = img.cuda()
            model = model.cuda()

        if cfg.feats:
            feats = model.features(img)
            feats = F.relu(feats, inplace=True)
            feats = F.adaptive_avg_pool2d(feats, (1, 1))
            output["feats"] = list(feats.cpu().detach().numpy().reshape(-1))

        preds = model(img).cpu()
        output["preds"] = dict(zip(xrv.datasets.default_pathologies,preds[0].detach().numpy()))

    if cfg.feats:
        print(output)
    else:
        pprint.pprint(output)

for i in pil_img_list:
    process_image(i)

usage: ipykernel_launcher.py [-h] [-f F] [-weights WEIGHTS] [-feats] [-cuda]
                             [-resize]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/thiagohata/Library/Jupyter/runtime/kernel-v2-1166XZuPmKAVlIZe.json


SystemExit: 2

In [ ]:
import torchxrayvision as xrv
import skimage, torch, torchvision

# Prepare the image:
img = skimage.io.imread("16747_3_1.jpg")
img = xrv.datasets.normalize(img, 255) # convert 8-bit image to [-1024, 1024] range
img = img.mean(2)[None, ...] # Make single color channel

transform = torchvision.transforms.Compose([xrv.datasets.XRayCenterCrop(),xrv.datasets.XRayResizer(224)])

img = transform(img)
img = torch.from_numpy(img)

# Load model and process image
model = xrv.models.DenseNet(weights="densenet121-res224-all")
outputs = model(img[None,...]) # or model.features(img[None,...]) 

# Print results
dict(zip(model.pathologies,outputs[0].detach().numpy()))

{'Atelectasis': 0.32797316,
 'Consolidation': 0.42933336,
 'Infiltration': 0.5316924,
 'Pneumothorax': 0.28849724,
 'Edema': 0.024142697,
 'Emphysema': 0.5011832,
 'Fibrosis': 0.51887786,
 'Effusion': 0.27805611,
 'Pneumonia': 0.18569896,
 'Pleural_Thickening': 0.24489835,
 'Cardiomegaly': 0.3645515,
 'Nodule': 0.68982,
 'Mass': 0.6392845,
 'Hernia': 0.00993878,
 'Lung Lesion': 0.011150705,
 'Fracture': 0.51916164,
 'Lung Opacity': 0.59073937,
 'Enlarged Cardiomediastinum': 0.27218717}
